In [56]:
import pandas as pd
import numpy as np

In [64]:
# load the CSV file exported from SPSS, and calculate modal split percentage for JFK passengers.
df = pd.read_csv('exploration/2014 TxT CSS O-D Dep SPSS Database_NYU w vars.csv')
df= df.query('AIRPORT=="JFK"')
# jfk_tot = df.query('AIRPORT=="JFK"').WEIGHT.sum()
# jfk_splits = df.query('AIRPORT=="JFK"').groupby('MAINMODE')['WEIGHT'].sum()/jfk_tot

# label each value of MAINMODE with a category
modegroups = {'A':'train',
'E':'train',
'J':'train',
'Z':'train',
'LIRR Connecting to JFK AirTrain':'train',
'Other (NYC Subway, including Connection to JFK AirTrain)':'train',
'Airlink New York':'van',
'All County Express':'van',
'Connecticut Limousine':'van',
"Dave's Best Limousine":'van',
'ETS Airport Shuttle':'van',
'Other (Shared-Ride Van/Service)':'van',
'Prime Time Shuttle of Connecticut':'van',
'State Shuttle':'van',
'SuperShuttle from Long Island':'van',
'SuperShuttle from Manhattan':'van',
'Limo/Executive Car/Town Car Service':'limo',
'Chartered/Tour Bus':'bus',
'NYC Airporter Bus from JFK/LaGuardia Airports':'bus',
'NYC Airporter Bus from Manhattan':'bus',
'Public/City Bus (that is, a local bus)':'bus',
'Transbridge Bus':'bus',
'Taxi':'taxi',
'Air Park':'local_van',
'Dollar Airport Parking':'local_van',
'Hilton Hotel':'local_van',
'Holiday Inn FSP':'local_van', 
'Hotel/Motel Shuttle/Van':'local_van',
'JFK Marriott Hotel':'local_van',
'National Airport Parking':'local_van',
'Other (Off-Airport Parking Shuttle/Van)':'local_van',
'Park Plus':'local_van',
'Sky Park/Central Parking':'local_van',
'The Parking Spot JFK/LGA':'local_van',
'U-Save':'local_van',
'Vista Airport Parking':'local_van',
'Drove Your Own Car':'private',
'Passenger in Car Parked at Airport':'private',
'Passenger in Car and Dropped Off at Airport':'private',
'Avis':'rental',
'Budget':'rental',
'Dollar':'rental',
'Enterprise':'rental',
'Hertz':'rental',
'National':'rental',
'Other (Drove Rental Car)':'rental'}

In [76]:
# label each value of MAINMODE with a category
modegroups = {'A':'transit',
'E':'transit',
'J':'transit',
'Z':'transit',
'LIRR Connecting to JFK AirTrain':'transit',
'Other (NYC Subway, including Connection to JFK AirTrain)':'transit',
'Airlink New York':'other',
'All County Express':'other',
'Connecticut Limousine':'other',
"Dave's Best Limousine":'other',
'ETS Airport Shuttle':'other',
'Other (Shared-Ride Van/Service)':'other',
'Prime Time Shuttle of Connecticut':'other',
'State Shuttle':'other',
'SuperShuttle from Long Island':'other',
'SuperShuttle from Manhattan':'other',
'Limo/Executive Car/Town Car Service':'other',
'Chartered/Tour Bus':'other',
'NYC Airporter Bus from JFK/LaGuardia Airports':'other',
'NYC Airporter Bus from Manhattan':'other',
'Public/City Bus (that is, a local bus)':'transit',
'Transbridge Bus':'other',
'Taxi':'taxi',
'Air Park':'other',
'Dollar Airport Parking':'other',
'Hilton Hotel':'other',
'Holiday Inn FSP':'other', 
'Hotel/Motel Shuttle/Van':'other',
'JFK Marriott Hotel':'other',
'National Airport Parking':'other',
'Other (Off-Airport Parking Shuttle/Van)':'other',
'Park Plus':'other',
'Sky Park/Central Parking':'other',
'The Parking Spot JFK/LGA':'other',
'U-Save':'other',
'Vista Airport Parking':'other',
'Drove Your Own Car':'other',
'Passenger in Car Parked at Airport':'other',
'Passenger in Car and Dropped Off at Airport':'other',
'Avis':'other',
'Budget':'other',
'Dollar':'other',
'Enterprise':'other',
'Hertz':'other',
'National':'other',
'Other (Drove Rental Car)':'other'}

In [77]:
# add groupings
# grouped_df = pd.DataFrame.from_dict(modegroups,orient='index').rename(columns={0:'ModeGroup'})
# summarize the mode-split percentages by mode-group
# grouped_df.join(jfk_splits).groupby('ModeGroup').sum()

In [78]:
zip_summary = pd.read_csv('zip_summary.csv')

In [79]:
survey_columns = df[['TOZip','MAINMODE','ODTravelTimeToAirport']]

In [80]:
survey_columns['zipcode'] = pd.to_numeric(survey_columns['TOZip'],errors='coerce')

C:\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [81]:
joined = survey_columns.dropna(how='any',subset=['zipcode']).merge(zip_summary,on='zipcode',how='inner')

In [82]:
joined.drop(['TOZip','taxi_mean_pickup_longitude','taxi_mean_pickup_latitude','taxi_mean_trip_distance'],axis=1,inplace=True)

In [83]:
joined = joined.merge(pd.DataFrame.from_dict(modegroups,orient='index').rename(columns={0:'ModeGroup'}),left_on='MAINMODE',right_index=True)

In [84]:
joined['taxi_mean_duration'] = np.divide(joined['taxi_mean_duration'],60)
joined['transit_duration'] = np.divide(joined['transit_duration'],60)

In [85]:
joined.to_csv('joined_jfk_responses.csv')

In [86]:
import statsmodels.api as st

In [88]:
joined['transit_fare'] = 7.75

In [100]:
y = joined.query('ModeGroup != "other"').dropna(how='any')['ModeGroup']
x = joined.query('ModeGroup != "other"').dropna(how='any')[['taxi_mean_duration','taxi_mean_total_w_inferred_tip','transit_duration','transit_fare']]

In [101]:
mdl = st.MNLogit(y, x)
mdl_fit = mdl.fit()

Optimization terminated successfully.
         Current function value: 0.578382
         Iterations 5


In [102]:
print mdl_fit.summary()

                          MNLogit Regression Results                          
Dep. Variable:              ModeGroup   No. Observations:                  183
Model:                        MNLogit   Df Residuals:                      179
Method:                           MLE   Df Model:                            3
Date:                Thu, 14 Apr 2016   Pseudo R-squ.:                 0.03091
Time:                        10:56:42   Log-Likelihood:                -105.84
converged:                       True   LL-Null:                       -109.22
                                        LLR p-value:                   0.08020
             ModeGroup=transit       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------------
taxi_mean_duration                -0.0639      0.032     -2.010      0.044        -0.126    -0.002
taxi_mean_total_w_inferred_tip     0.0724      0.029      2.486      0.

In [103]:
mdl_margeff = mdl_fit.get_margeff()
print mdl_margeff.summary()

       MNLogit Marginal Effects      
Dep. Variable:              ModeGroup
Method:                          dydx
At:                           overall
                ModeGroup=taxi      dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------------
taxi_mean_duration                 0.0125      0.006      2.087      0.037         0.001     0.024
taxi_mean_total_w_inferred_tip    -0.0142      0.005     -2.626      0.009        -0.025    -0.004
transit_duration                   0.0027      0.003      1.020      0.308        -0.002     0.008
--------------------------------------------------------------------------------------------------
             ModeGroup=transit      dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------------
taxi_mean_duration                -0.0125      0.006    